<a href="https://colab.research.google.com/github/ManishSuhas0026/LLM/blob/main/Manish_530_Lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [3]:
from datasets import load_dataset
from transformers import BertTokenizerFast, BertForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification

dataset = load_dataset("conll2003", trust_remote_code= True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [19]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples['tokens'],
        truncation=True,
        padding=True,
        max_length=64,
        is_split_into_words=True
    )

    labels = []
    for i, label in enumerate(examples['ner_tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [13]:
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

In [14]:
tokenized_dataset = tokenized_dataset.remove_columns(["tokens", "pos_tags", "chunk_tags", "ner_tags"])

In [15]:
tokenized_dataset.set_format("torch")

In [16]:
small_train_dataset = tokenized_dataset['train'].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_dataset['validation'].shuffle(seed=42).select(range(500))

In [17]:
model = BertForTokenClassification.from_pretrained('bert-base-cased', num_labels=len(dataset['train'].features['ner_tags'].feature.names))

data_collator = DataCollatorForTokenClassification(tokenizer)

# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

results = trainer.evaluate()
print(results)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.257827
2,No log,0.125247
3,No log,0.110404


{'eval_loss': 0.11040418595075607, 'eval_runtime': 1.0136, 'eval_samples_per_second': 493.291, 'eval_steps_per_second': 31.571, 'epoch': 3.0}


In [8]:
results = trainer.evaluate()
print(results)

{'eval_loss': 0.11891372501850128, 'eval_runtime': 0.6039, 'eval_samples_per_second': 827.996, 'eval_steps_per_second': 52.992, 'epoch': 3.0}


In [18]:
import torch
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')

sentences = [
    "Hugging Face Inc. is a company based in New York City.",
    "Bert is a neural network-based technique for natural language processing."
]

tokenized_inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

input_ids = tokenized_inputs["input_ids"].to(device)
attention_mask = tokenized_inputs["attention_mask"].to(device)

with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)

logits = outputs.logits

predictions = torch.argmax(logits, dim=2).cpu().numpy()

label_names = dataset['train'].features['ner_tags'].feature.names

def align_predictions(predictions, label_ids):
    preds = []
    for pred in predictions:
        preds.append([label_names[p] for p in pred])
    return preds

aligned_predictions = align_predictions(predictions, input_ids.cpu().numpy())

for sentence, tokens, preds in zip(sentences, tokenized_inputs["input_ids"], aligned_predictions):
    print(f"Sentence: {sentence}")
    tokenized_tokens = tokenizer.convert_ids_to_tokens(tokens)
    print("Tokenized Tokens:", tokenized_tokens)
    print("Predictions:", preds)

Sentence: Hugging Face Inc. is a company based in New York City.
Tokenized Tokens: ['[CLS]', 'Hu', '##gging', 'Face', 'Inc', '.', 'is', 'a', 'company', 'based', 'in', 'New', 'York', 'City', '.', '[SEP]']
Predictions: ['O', 'O', 'O', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'I-LOC', 'I-LOC', 'O', 'O']
Sentence: Bert is a neural network-based technique for natural language processing.
Tokenized Tokens: ['[CLS]', 'Bert', 'is', 'a', 'neural', 'network', '-', 'based', 'technique', 'for', 'natural', 'language', 'processing', '.', '[SEP]', '[PAD]']
Predictions: ['O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
